### Go through ... 
1. County deaths
1. State (NC) projections
1. County json
1. County mobility (later lmao)

# NC county deaths

In [1]:
import pandas as pd

df = pd.read_csv('raw_data/nyt_us-counties-2021.csv')
display(df.head(), len(df))

dff=df[df.state=='North Carolina']
display(dff.head(), len(dff))

dff.to_csv('processed_data/nyt_nc_counties.csv',index=False)

,date,county,state,fips,cases,deaths
0,2021-01-01,Autauga,Alabama,1001.0,4239,50.0
1,2021-01-01,Baldwin,Alabama,1003.0,13823,169.0
2,2021-01-01,Barbour,Alabama,1005.0,1517,33.0
3,2021-01-01,Bibb,Alabama,1007.0,1854,46.0
4,2021-01-01,Blount,Alabama,1009.0,4693,63.0


1185373

,date,county,state,fips,cases,deaths
1903,2021-01-01,Alamance,North Carolina,37001.0,10397,117.0
1904,2021-01-01,Alexander,North Carolina,37003.0,2662,30.0
1905,2021-01-01,Alleghany,North Carolina,37005.0,638,2.0
1906,2021-01-01,Anson,North Carolina,37007.0,1407,26.0
1907,2021-01-01,Ashe,North Carolina,37009.0,1212,27.0


36501

In [2]:
dff[dff.fips==37007]

,date,county,state,fips,cases,deaths
1906,2021-01-01,Anson,North Carolina,37007.0,1407,26.0
5153,2021-01-02,Anson,North Carolina,37007.0,1464,27.0
8399,2021-01-03,Anson,North Carolina,37007.0,1482,27.0
11645,2021-01-04,Anson,North Carolina,37007.0,1495,27.0
14890,2021-01-05,Anson,North Carolina,37007.0,1519,27.0
...,...,...,...,...,...,...
1171033,2021-12-27,Anson,North Carolina,37007.0,4144,85.0
1174282,2021-12-28,Anson,North Carolina,37007.0,4204,85.0
1177531,2021-12-29,Anson,North Carolina,37007.0,4217,85.0
1180781,2021-12-30,Anson,North Carolina,37007.0,4249,85.0


# NC (state) projections

In [3]:
# Need to read all files ...

import os

ddir = 'raw_data/covidhub-trained_ensemble-2021/'

df_list = []
for fname in os.listdir(ddir):
    if fname.endswith('.csv'):
        fpath = os.path.join(ddir, fname)
        df = pd.read_csv(fpath)
        df = df[df.location=='37']
        df = df[df.target.isin(['1 wk ahead cum death',
                        '2 wk ahead cum death',
                        '3 wk ahead cum death',
                        '4 wk ahead cum death'])]
        df_list.append(df)
#         print(fname, len(df))

In [4]:
bigdf = pd.concat(df_list)
bigdf.to_csv('processed_data/covidhub_projections.csv',index=False)
bigdf

,forecast_date,target,target_end_date,location,type,quantile,value
33,2021-02-01,1 wk ahead cum death,2021-02-06,37,quantile,0.010,9620
89,2021-02-01,2 wk ahead cum death,2021-02-13,37,quantile,0.010,10034
145,2021-02-01,3 wk ahead cum death,2021-02-20,37,quantile,0.010,10391
201,2021-02-01,4 wk ahead cum death,2021-02-27,37,quantile,0.010,10712
257,2021-02-01,1 wk ahead cum death,2021-02-06,37,quantile,0.025,9654
...,...,...,...,...,...,...,...
5130,2021-12-27,4 wk ahead cum death,2022-01-22,37,quantile,0.990,20660
5186,2021-12-27,1 wk ahead cum death,2022-01-01,37,point,NaN,19372
5242,2021-12-27,2 wk ahead cum death,2022-01-08,37,point,NaN,19568
5298,2021-12-27,3 wk ahead cum death,2022-01-15,37,point,NaN,19828


# NC county covariates

In [20]:
import json

with open('raw_data/county_covariates.json') as f:
    d = json.load(f)
print(len(d))

3142


In [21]:
nc_covs = []
for entry in d:
    if entry['state']=='NC':
        nc_covs.append(entry)
print(len(nc_covs))

100


In [24]:
nc_covs[0].keys()

dict_keys(['name', 'fips', 'state', 'land_area (km^2)', 'area (km^2)', 'longitude (deg)', 'latitude (deg)', 'noaa', 'zip-codes', 'race', 'age', 'male', 'female', 'population', 'deaths', 'bls', 'life-expectancy', 'fatal_police_shootings', 'police_deaths', 'avg_income', 'covid-deaths', 'covid-confirmed', 'covid-vaccination', 'elections', 'edu', 'poverty-rate', 'cost-of-living', 'industry', 'health'])

In [37]:
# for k,v in nc_covs[0].items():
# #     print(k,type(v))
#     if isinstance(v,dict):
#         print(k,v)
#         print('')
#     elif isinstance(v,list):
#         print(k,v)
#         print('')

In [40]:
to_drop = []
for k,v in nc_covs[0].items():
    if isinstance(v,dict) or isinstance(v,list):
        to_drop.append(k)

# nc_covs_filtered = {k:v for k,v in nc_covs[0].items() if k not in to_drop}
# (nc_covs_filtered)

In [49]:
nc_covs_filtered = []

for cov in nc_covs:
    cov_filtered = {k:v for k,v in cov.items() if k not in to_drop}
    nc_covs_filtered.append(cov_filtered)
    
import csv

keys = nc_covs_filtered[0].keys()
with open('processed_data/nc_county_covs.csv', 'w', newline='') as out:
    dict_writer = csv.DictWriter(out, keys)
    dict_writer.writeheader()
    dict_writer.writerows(nc_covs_filtered)

In [48]:
# nc_covs_filtered